<a href="https://colab.research.google.com/github/jonas-richter/tara_guidi2016_data/blob/data_exploration/Eukaryote_abundance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eukaryotes count per sample

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import plotly.express as px
sns.set()
## ML
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
# read data
euk_norm = pd.read_csv('https://github.com/jonas-richter/tara_guidi2016_data/blob/data_exploration/data/euk.norm.ab.tsv?raw=true',
                  on_bad_lines='skip', sep='\t', header = 0)

In [4]:
# view data
euk_norm.loc[[0,1,2,3,4,5]]

,Sample,Metazoa_Morula+musiva,Spumellarida_Actinomma+boreale,Basidiomycota_Malassezia+slooffiae,Cryptophyta_Rhodomonas+sp.,Metazoa_Ircinia+felix_f._felix,Ascomycota_Cladosporium+cladosporioides,Microsporidiomycota_Ovipleistophora+mirandellae,Collodaria_Sphaerozoum+punctatum,MALV-II_MALV-II-Clade-29_X+sp.,...,Ciliophora_Tintinnopsis+dadayi,Pelagophyceae_Pelagophyceae_XXX+sp.,Pinguiophyceae_Pinguiochrysis+pyriformis,Ciliophora_Tintinnopsis+lacustris,Metazoa_Corynactis,Metazoa_Gastrochaena+dubia,Choanoflagellida_Salpingoeca+urceolata,Bacillariophyta_Corethron+inerme,Entomophthoromycota_Entomophthora+schizophorae,Haptophyta_Calyptrosphaera+radiata
0,S4DCM,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000059,0.0,...,0.0,0.000050,0.000000e+00,0.0,5.987868e-07,0.000000,0.000000e+00,1.080810e-04,0.0,0.0
1,S4SUR,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000755,0.0,...,0.0,0.000025,0.000000e+00,0.0,0.000000e+00,0.000000,0.000000e+00,5.214899e-07,0.0,0.0
2,S7DCM,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000004,0.0,...,0.0,0.000005,0.000000e+00,0.0,0.000000e+00,0.000012,9.949298e-07,0.000000e+00,0.0,0.0
3,S7SUR,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000195,0.0,...,0.0,0.000027,4.726870e-07,0.0,0.000000e+00,0.000016,7.090304e-07,0.000000e+00,0.0,0.0
4,S9DCM,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.0,3.425431e-07,0.000010,0.0,...,0.0,0.000140,0.000000e+00,0.0,2.264210e-04,0.000000,0.000000e+00,0.000000e+00,0.0,0.0
5,S9SUR,0.0,6.622782e-07,0.0,6.622782e-07,0.0,0.0,0.000000e+00,0.004247,0.0,...,0.0,0.000077,2.582885e-05,0.0,6.622782e-07,0.000000,0.000000e+00,0.000000e+00,0.0,0.0


In [5]:
euk_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Columns: 1762 entries, Sample to Haptophyta_Calyptrosphaera+radiata
dtypes: float64(1761), object(1)
memory usage: 922.4+ KB


In [8]:
# make Eukaryotic lineages as row name
euk_norm_2 = euk_norm.set_index('Sample')

# plot
px.imshow(euk_norm_2, aspect = 0.2)